In [1]:
import numpy as np
import math
import random

In [2]:
def generate_binary_array(num_centroids):
    # Determine k from num_centroids
    k = int(np.log2(num_centroids))

    # Generate all possible binary combinations of length k
    b = np.array([list(map(int, np.binary_repr(i, width=k))) for i in range(num_centroids)])

    return b

In [3]:
def channel_with_memory(num_level, epsilon, delta):
    Pr = np.zeros((num_level, num_level))
    n = int(np.log2(num_level))

    # Transition probability matrix for the binary symmetric channel with memory
    Pr_z = np.array([
        [(1 - epsilon + delta) / (1 + delta), epsilon / (1 + delta)],
        [(1 - epsilon) / (1 + delta), (epsilon + delta) / (1 + delta)]
    ])

    for x in range(num_level):
        for y in range(num_level):
            binary_x = np.array([int(bit) for bit in np.binary_repr(x, width=n)])
            binary_y = np.array([int(bit) for bit in np.binary_repr(y, width=n)])
            binary_z = binary_x ^ binary_y  # XOR operation

            if binary_z[0] == 1:
                probability = epsilon
            else:
                probability = 1 - epsilon
            for i in range(1, n):
                probability *= Pr_z[binary_z[i - 1], binary_z[i]]

            Pr[x, y] = probability

    return Pr

In [4]:
# Simulated Annealing Algorithm
def simulated_annealing(T_0, alpha, T_f, b, N_fail, N_success, N_cut, k, epsilon, delta, centroids, partitions, num_centroids):
    T = T_0
    count = 0
    count_success = 0
    count_fail = 0
    b_history = []

    prob_points = []

    # Loop over each partition
    for partition in partitions:
        # Calculate the probability of samples falling in this partition
        prob = len(partition) / 500000
        prob_points.append(prob)

    conditional_prob = channel_with_memory(num_centroids, epsilon, delta)

    while T > T_f and count_fail < N_fail:

        b_prime = random.sample(b, len(b))

        delta_Dc = 0

        distortion_b = 0
        distortion_b_prime = 0

        for g in range(0, num_centroids):
            for h in range(0, num_centroids):
                distortion_b = distortion_b + prob_points[g] * conditional_prob[h,b[g]] * ((centroids[b[g]]-centroids[h])**2)
                distortion_b_prime = distortion_b_prime + prob_points[g] * conditional_prob[h, b_prime[g]] * ((centroids[b_prime[g]]-centroids[h])**2)

        distortion_b = distortion_b * (1 / k)
        distortion_b_prime = distortion_b_prime * (1 / k)
        delta_Dc = distortion_b_prime - distortion_b
        b_history.append(distortion_b)

        if delta_Dc <= 0:
            b = b_prime
            count_success = count_success + 1
            count_fail = 0
        else:
            rand_num = random.uniform(0, 1)
            if rand_num <= math.exp(-delta_Dc / T):
                b = b_prime
            count_fail = count_fail + 1

        if count >= N_cut or count_success >= N_success:
            T = alpha * T
            count = 0
            count_success = 0
        count = count + 1

    return b

In [5]:
# Function to generate normalized source signal that will be used for training
def generate_source_signal(distribution, num_samples=500000):

    if distribution.lower() == 'laplace':
        source = np.random.laplace(loc=0, scale=np.sqrt(1/2), size=num_samples)
    else:
        source = np.random.normal(loc=0, scale=1, size=num_samples)

    # Normalize (zero-mean, unit variance)
    source = (source - np.mean(source)) / np.std(source)

    return source

In [6]:
def generate_initial_codebook(source, num_centroids):
    min_samples = np.min(source)
    max_samples = np.max(source)
    width = (max_samples - min_samples) / num_centroids
    centroids = []
    for i in range(num_centroids):
        # Calculate the current centroid
        centroid_current = min_samples + (i + 0.5) * width
        centroids.append(centroid_current)

    return centroids

In [26]:
def cosq_design(source, current_codebook, epsilon, b_obtained, tol=1e-4, max_iter=100):
    n_codewords = len(current_codebook)
    P_Y_given_X = channel_with_memory(n_codewords, epsilon, 10)

    print(P_Y_given_X)

    # Initialize codebook (ENSURE IT'S A NUMPY ARRAY)
    codebook = np.asarray(current_codebook.copy())  # Convert to NumPy array

    signal_power = np.mean(source ** 2)

    for iteration in range(max_iter):
        # --------------------------------------------------
        # Generalized NNC (Nearest Neighbor Condition)
        # --------------------------------------------------

        # Initialize the partitions
        partitions = [[] for _ in range(n_codewords)]

        for v in source:
          distortions = []

          # Iterate over each partition index
          for i in range(num_centroids):
            # Assuming v is assigned to i
            distortion = 0

            for j in range(num_centroids):
              # Compute the total distortion assuming that v is assigned to partition[i]
              distortion += P_Y_given_X[j, b_obtained[i]] * ((v - codebook[j])**2)

            distortions.append(distortion)

          # To find the minimum i from the distortion list
          min_distortion_idx = np.argmin(distortions)
          # Add v to that partition
          partitions[min_distortion_idx].append(v)

        # To visualize the partition of the line
        print(len(partitions[0]),len(partitions[1]),len(partitions[2]),len(partitions[3]))
        # --------------------------------------------------
        # Generalized CC (Centroid Condition)
        # --------------------------------------------------
        new_codebook = np.zeros_like(codebook)
        for i in range(n_codewords):
            numerator = 0.0
            denominator = 0.0

            for j in range(n_codewords):
                # If there is no element in that partition, skip that partition set
                if len(partitions[j]) == 0:
                    continue


                prob = P_Y_given_X[i, b_obtained[j]]
                sum_v = np.sum(partitions[j])
                count = len(partitions[j])

                numerator += prob * sum_v
                denominator += prob * count

            new_codebook[i] = numerator / denominator




        print(new_codebook)
        # Check convergence
        codebook_change = np.max(np.abs(new_codebook - codebook))
        codebook = new_codebook.copy()

        if codebook_change < tol:
            break


    # Calculate MSE
    mse_q = 0
    # For partition i
    for i in range(n_codewords):
        for x in partitions[i]:
            mse_q = mse_q + (x - codebook[i])**2
    mse_q = mse_q /len(source)

    mse_c = 0
    for i in range(n_codewords):
        for k in range(n_codewords):

            prod = P_Y_given_X[i ,b_obtained[k]]*(codebook[i]-codebook[k])**2
            mse_c += prod

    mse_c = mse_c / n_codewords

    # Final SNR calculation (SDR)
    noise_distortion = mse_q + mse_c
    snr = 10 * np.log10(signal_power / noise_distortion)

    return codebook, partitions, snr

In [8]:
#--------------------------------------------------------------
# Parameter declarations
#--------------------------------------------------------------

num_centroids = 4
# For SA
T_0 = 10
alpha = 0.97
T_f = 0.00025
N_fail = 50000
N_success = 5
N_cut = 200
k = 10
# For channel
delta = 10

initial_distribution = 'Gaussian'

sampled_source = generate_source_signal(initial_distribution)

codebooks_set = []

# Create an empty array b with length num_centroids
init_b = [0] * num_centroids

# Populate the array with values from 0 to num_centroids - 1
for i in range(num_centroids):
    init_b[i] = i


In [27]:
# COSQ for noiseless. Works for a preset low noise epsilon
init_codebook = generate_initial_codebook(sampled_source, num_centroids)
codebooks_set.append(np.array(init_codebook))

print(init_codebook)
noiseless_codebook, noiseless_partition, noiseless_snr = cosq_design(sampled_source, codebooks_set[-1],  1e-11, init_b)
print(noiseless_snr)
codebooks_set.append(np.array(noiseless_codebook))

[np.float64(-3.613810675414047), np.float64(-1.1810278979727529), np.float64(1.2517548794685407), np.float64(3.6845376569098356)]
[[1.00000000e+00 9.09090909e-13 9.09090909e-13 9.09090909e-12]
 [9.09090909e-13 1.00000000e+00 9.09090909e-12 9.09090909e-13]
 [9.09090909e-13 9.09090909e-12 1.00000000e+00 9.09090909e-13]
 [9.09090909e-12 9.09090909e-13 9.09090909e-13 1.00000000e+00]]
4180 253070 239400 3350
[-2.72768999 -0.74292448  0.79391889  2.79088399]
20601 234680 226399 18320
[-2.14441296 -0.6617371   0.7040228   2.18796158]
39911 214528 208548 37013
[-1.85906542 -0.58405395  0.62073279  1.89232174]
55231 198666 193834 52269
[-1.70632078 -0.52984394  0.56185061  1.73330327]
65716 187706 183616 62962
[-1.62054106 -0.49554335  0.52322996  1.64287259]
72445 180549 177159 69847
[-1.57102507 -0.47469331  0.4990111   1.59082001]
76576 176085 173162 74177
[-1.54238422 -0.46235073  0.48395587  1.56005064]
79049 173346 170734 76871
[-1.52580934 -0.45522819  0.47454966  1.5415947 ]
80468 17168

In [28]:
# Once we obtain the noiseless_codebook, we perform SA algorithm to obtain the shuffling before we train with noisy channels
b_from_sa = simulated_annealing(T_0, alpha, T_f, init_b, N_fail, N_success, N_cut, k, 0.005, delta, noiseless_codebook, noiseless_partition, num_centroids)

print(b_from_sa)

[0, 1, 2, 3]


In [29]:
# Starting with noiseless codebook and b obtained from SA algorithm, run the training for epsilon = 0.005
codebook_1, partition_1, snr_1 = cosq_design(sampled_source, codebooks_set[-1], 0.005, b_from_sa)
print(snr_1)
codebooks_set.append(np.array(codebook_1))

[[9.94547727e-01 4.52272727e-04 4.52272727e-04 4.54772727e-03]
 [4.52272727e-04 9.94547727e-01 4.54772727e-03 4.52272727e-04]
 [4.52272727e-04 4.54772727e-03 9.94547727e-01 4.52272727e-04]
 [4.54772727e-03 4.52272727e-04 4.52272727e-04 9.94547727e-01]]
80733 169374 168848 81045
[-1.49806688 -0.45174662  0.45072447  1.49738925]
81426 168665 168158 81751
[-1.49364798 -0.44963177  0.44847503  1.49290901]
81847 168229 167738 82186
[-1.490977   -0.44836324  0.4470779   1.49016327]
82099 167960 167508 82433
[-1.48938498 -0.44762535  0.44626348  1.48860744]
82256 167795 167366 82583
[-1.48839544 -0.44715898  0.44577563  1.48766349]
82336 167711 167263 82690
[-1.48788807 -0.44692227  0.44543109  1.48699495]
82389 167650 167217 82744
[-1.48755419 -0.446779    0.44524189  1.48665629]
82420 167614 167188 82778
[-1.48735869 -0.44669614  0.4451228   1.48644341]
82429 167600 167174 82797
[-1.48730081 -0.44668176  0.44505014  1.48632549]
82435 167590 167170 82805
[-1.48726281 -0.44667377  0.44501462 

In [30]:
codebook_2, partition_2, snr_2 = cosq_design(sampled_source, codebooks_set[-1], 0.01, b_from_sa)
print(snr_2)
codebooks_set.append(np.array(codebook_2))

[[9.891e-01 9.000e-04 9.000e-04 9.100e-03]
 [9.000e-04 9.891e-01 9.100e-03 9.000e-04]
 [9.000e-04 9.100e-03 9.891e-01 9.000e-04]
 [9.100e-03 9.000e-04 9.000e-04 9.891e-01]]
81404 168619 168176 81801
[-1.47722013 -0.44556122  0.44379723  1.47618421]
82124 167894 167486 82496
[-1.47275327 -0.44335425  0.44163781  1.47187708]
82556 167465 167038 82941
[-1.47008036 -0.44201882  0.44027188  1.46913984]
82815 167205 166755 83225
[-1.46847864 -0.44122748  0.439392    1.46740338]
82963 167047 166592 83398
[-1.4675625  -0.44080066  0.43883137  1.46635011]
83046 166954 166511 83489
[-1.4670506  -0.44057235  0.43852469  1.46579544]
83083 166912 166453 83552
[-1.46681713 -0.44047356  0.43831599  1.46541655]
83101 166882 166423 83594
[-1.46670084 -0.44045101  0.43815396  1.4651657 ]
83113 166858 166404 83625
[-1.46662261 -0.44044651  0.43802616  1.46498091]
83115 166849 166389 83647
[-1.46660554 -0.44045984  0.43793893  1.46485136]
83116 166836 166395 83653
[-1.4665982  -0.4404881   0.43788905  1.4

In [31]:
codebook_3, partition_3, snr_3 = cosq_design(sampled_source, codebooks_set[-1], 0.05, b_from_sa)
print(snr_3)
codebooks_set.append(np.array(codebook_3))


[[0.94568182 0.00431818 0.00431818 0.04568182]
 [0.00431818 0.94568182 0.04568182 0.00431818]
 [0.00431818 0.04568182 0.94568182 0.00431818]
 [0.04568182 0.00431818 0.00431818 0.94568182]]
74368 175577 175144 74911
[-1.38472239 -0.43141853  0.42869969  1.38426422]
80320 169616 169249 80815
[-1.35205868 -0.41411561  0.41148649  1.35178216]
83738 166206 165827 84229
[-1.33392507 -0.40431635  0.40170532  1.33368518]
85596 164348 163864 86192
[-1.32412718 -0.39908656  0.39608267  1.32359577]
86634 163268 162830 87268
[-1.31872797 -0.39626623  0.39292927  1.31810198]
87236 162646 162228 87890
[-1.31561799 -0.3946242   0.39112133  1.31494182]
87552 162309 161891 88248
[-1.31395817 -0.39379685  0.3900512   1.31316077]
87737 162105 161695 88463
[-1.31298236 -0.39332947  0.38939338  1.31209822]
87834 161987 161613 88566
[-1.31248147 -0.39310531  0.38905521  1.31158096]
87886 161924 161571 88619
[-1.31221543 -0.39298385  0.38888158  1.31131268]
87918 161884 161549 88649
[-1.3120545  -0.39291091 

In [32]:
codebook_4, partition_4, snr_4 = cosq_design(sampled_source, codebooks_set[-1], 0.1, b_from_sa)
print(snr_4)
codebooks_set.append(np.array(codebook_4))

[[0.89181818 0.00818182 0.00818182 0.09181818]
 [0.00818182 0.89181818 0.09181818 0.00818182]
 [0.00818182 0.09181818 0.89181818 0.00818182]
 [0.09181818 0.00818182 0.00818182 0.89181818]]
75556 174235 173963 76246
[-1.21171192 -0.38265707  0.37864361  1.21307794]
83679 166136 165782 84403
[-1.17598531 -0.36151339  0.35741127  1.17710534]
88001 161801 161385 88813
[-1.15744723 -0.35060094  0.3460662   1.15846359]
90341 159425 159081 91153
[-1.1476829  -0.34477331  0.34009258  1.14864692]
91558 158195 157801 92446
[-1.14251232 -0.34179861  0.33677124  1.14341333]
92215 157502 157153 93130
[-1.13975977 -0.34023927  0.33497784  1.14063288]
92543 157150 156781 93526
[-1.13829066 -0.33950825  0.333901    1.13912181]
92722 156940 156583 93755
[-1.13746707 -0.33914906  0.33324211  1.13826818]
92849 156782 156481 93888
[-1.13693675 -0.33889036  0.33285233  1.13773142]
92898 156721 156417 93964
[-1.13668783 -0.33880688  0.33262396  1.13746708]
92929 156677 156393 94001
[-1.13655036 -0.3387561  

In [33]:
codebook_5, partition_5, snr_5 = cosq_design(sampled_source, codebooks_set[-1], 0.05, b_from_sa)
print(snr_5)
codebooks_set.append(np.array(codebook_5))

[[0.94568182 0.00431818 0.00431818 0.04568182]
 [0.00431818 0.94568182 0.04568182 0.00431818]
 [0.00431818 0.04568182 0.94568182 0.00431818]
 [0.04568182 0.00431818 0.00431818 0.94568182]]
105065 144531 144611 105793
[-1.2293535  -0.3464751   0.34125711  1.22862184]
97370 152326 152112 98192
[-1.26529187 -0.36705102  0.36203596  1.26422899]
93159 156558 156296 93987
[-1.28572838 -0.37855255  0.37366138  1.28459614]
90897 158834 158634 91635
[-1.29702648 -0.3847536   0.38025858  1.29613974]
89577 160191 159920 90312
[-1.30364797 -0.38834692  0.38403627  1.30275849]
88876 160905 160620 89599
[-1.30719933 -0.39027194  0.38606723  1.30633808]
88472 161316 161048 89164
[-1.30927832 -0.39137642  0.38731464  1.30850554]
88243 161565 161252 88940
[-1.3104361  -0.39197571  0.38798622  1.3096459 ]
88103 161714 161365 88818
[-1.31112936 -0.39235432  0.38834291  1.31028347]
88032 161783 161433 88752
[-1.3114855  -0.3925593   0.38852254  1.31062367]
87995 161817 161466 88722
[-1.31166653 -0.3926719

In [34]:
codebook_6, partition_6, snr_6 = cosq_design(sampled_source, codebooks_set[-1], 0.01, b_from_sa)
print(snr_6)
codebooks_set.append(np.array(codebook_6))

[[9.891e-01 9.000e-04 9.000e-04 9.100e-03]
 [9.000e-04 9.891e-01 9.100e-03 9.000e-04]
 [9.000e-04 9.100e-03 9.891e-01 9.000e-04]
 [9.100e-03 9.000e-04 9.000e-04 9.891e-01]]
96520 153283 152996 97201
[-1.38851735 -0.40068629  0.39668363  1.38644145]
90768 159048 158730 91454
[-1.42100794 -0.41769541  0.41382482  1.4187012 ]
87461 162369 161961 88209
[-1.44033397 -0.42759974  0.42364358  1.43756474]
85548 164272 163832 86348
[-1.45175189 -0.43342236  0.42927428  1.44860369]
84487 165307 164990 85216
[-1.45818246 -0.43671534  0.43267107  1.45537775]
83855 165957 165606 84582
[-1.46203034 -0.43859908  0.43465954  1.45921025]
83514 166307 165977 84202
[-1.46412293 -0.43961695  0.4358517   1.46150745]
83324 166515 166167 83994
[-1.46529057 -0.4401514   0.43653856  1.46276857]
83212 166642 166276 83870
[-1.4659801  -0.44045517  0.43695941  1.46352108]
83147 166715 166345 83793
[-1.4663818  -0.44063299  0.43721775  1.46398767]
83119 166757 166363 83761
[-1.46655462 -0.44068244  0.43735271  1.4

In [35]:
codebook_7, partition_7, snr_7 = cosq_design(sampled_source, codebooks_set[-1], 0.005, b_from_sa)
print(snr_7)
codebooks_set.append(np.array(codebook_7))

[[9.94547727e-01 4.52272727e-04 4.52272727e-04 4.54772727e-03]
 [4.52272727e-04 9.94547727e-01 4.54772727e-03 4.52272727e-04]
 [4.52272727e-04 4.54772727e-03 9.94547727e-01 4.52272727e-04]
 [4.54772727e-03 4.52272727e-04 4.52272727e-04 9.94547727e-01]]
84041 165853 165432 84674
[-1.47719353 -0.44206212  0.43886111  1.47469017]
83349 166542 166126 83983
[-1.48150202 -0.44420596  0.44099262  1.47896375]
82965 166926 166530 83579
[-1.48390592 -0.44539301  0.44224671  1.48147113]
82738 167157 166775 83330
[-1.48533164 -0.44608477  0.44303131  1.48301987]
82610 167289 166916 83185
[-1.48613742 -0.4464703   0.44349306  1.48392278]
82532 167372 166996 83100
[-1.48662848 -0.44669869  0.44377075  1.48445296]
82485 167426 167054 83035
[-1.48692624 -0.44682534  0.44399198  1.48485713]
82452 167474 167080 82994
[-1.48713482 -0.44688787  0.44415936  1.48511264]
82432 167504 167100 82964
[-1.4872619  -0.44692319  0.44427941  1.48529917]
82422 167522 167117 82939
[-1.48732668 -0.4469326   0.44437881 